# Tutorial goal

This tutorial aims to show how to **configure** a **test environment** using
the **TestEnv module** provided by LISA.

# Configure logging

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2018-07-10 16:13:13,362 INFO    : root         : Using LISA logging configuration:
2018-07-10 16:13:13,363 INFO    : root         :   /home/wz/dev/linaro-dev/lisa/logging.conf


In [2]:
# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [3]:
# Other python modules required by this notebook
import json
import time
import os

# Test environment setup

## Do you have custom scripts to deploy and use on target?

In [4]:
# Custom scrips must be deployed under $LISA_HOME/tools
!tree ../../tools

../../tools
├── arm64
│   ├── perf
│   ├── README.rt-app
│   ├── README.taskset
│   ├── README.trace-cmd
│   ├── rt-app
│   ├── sysbench
│   ├── taskset
│   └── trace-cmd
├── armeabi
│   ├── perf
│   ├── README.rt-app
│   ├── rt-app
│   ├── sysbench
│   ├── taskset
│   └── trace-cmd
├── LICENSE.perf
├── LICENSE.rt-app
├── LICENSE.sysbench
├── LICENSE.taskset
├── LICENSE.trace-cmd
├── plots.py
├── report.py
├── scripts
│   ├── cgroup_run_into.sh
│   ├── cgroup_tasks_move.sh
│   ├── cpuidle_sampling.sh
│   ├── odroid_sampler.py
│   ├── odroid_sampler.sh
│   └── trace_frequencies.sh
├── wa_user_directory
│   ├── config.yaml
│   └── plugins
│       └── ubsystemuijanktests
│           └── __init__.py
├── wltests
│   ├── agendas
│   │   ├── example-exoplayer-simple.yaml
│   │   ├── example-jankbench.yaml
│   │   ├── example-rich.yaml
│   │   ├── sched-evaluation-full-traced.yaml
│   │   ├── sched-evaluation-full.yaml
│   │   └── sched-evaluation-small.yaml
│   ├── android
│   │   ├── create_

In [5]:
# This is the (not so fancy) script we want to deploy
!cat ../../tools/scripts/cpuidle_sampling.sh

#!/bin/sh

# CPU to monitor
CPU=${1:-0}
# Sampling time
SLEEP=${2:-1}
# Samples to collect
COUNT=${3:-3}

# Enter CPU's sysfs
cd /sys/devices/system/cpu

# Initial C-State residencies counter
ISC=$(find cpu0/cpuidle -name "state*" | wc -l)
for I in $(seq 0 $((ISC-1))); do
	LCS[$I]=`cat cpu$CPU/cpuidle/state$I/usage`
done

# Dump header
printf "#%13s " "Time"
for I in $(seq 0 $((ISC-1))); do
  printf "%14s " "idle$I"
done
echo

# Sampling loop
for I in $(seq $COUNT); do

	sleep $SLEEP

	# Dump CPU C-State residencies
	now=$(date +%s)
	printf "%14d " $now
	for I in $(seq 0 $((ISC-1))); do
		U=`cat cpu$CPU/cpuidle/state$I/usage`
		CCS=$(($U - ${LCS[$I]}))
		printf "%14d " $CCS
		LCS[$I]=$U
	done
	echo


done

# vim: ts=2


## Which devlib modules you need for your experiments?

In [6]:
# You can have a look at the devlib supported modules by lising the
devlib_modules_folder = 'libs/devlib/devlib/module/'

logging.info("Devlib provided modules are found under:")
logging.info("   $LISA_HOME/{}".format(devlib_modules_folder))
!cd ../../ ; find {devlib_modules_folder} -name "*.py" | sed 's|libs/devlib/devlib/module/|   |' | grep -v __init__ 

2018-07-10 16:13:13,916 INFO    : root         : Devlib provided modules are found under:
2018-07-10 16:13:13,918 INFO    : root         :    $LISA_HOME/libs/devlib/devlib/module/


   cgroups.py
   hwmon.py
   biglittle.py
   hotplug.py
   android.py
   cpufreq.py
   thermal.py
   cooling.py
   gem5stats.py
   vexpress.py
   gpufreq.py
   cpuidle.py


## Setup you TestEnv confguration

In [7]:
# Setup a target configuration
conf = {

    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # platform type, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'sp9863a',   # board type, valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
    "device"      : 'SC98631A10183230316',
    # Login credentials
    # "host"        : "192.168.0.1",
    # "username"    : "root",
    # "password"    : "",

    # Custom tools to deploy on target, they must be placed under:
    #   $LISA_HOME/tools/(ARCH|scripts)
    "tools" : [ "cpuidle_sampling.sh" ],

    # FTrace configuration
    "ftrace" : {
         "events" : [
             "cpu_idle",
             "sched_switch",
         ],
         "buffsize" : 10240,
    },
    
    # Where results are collected
    "results_dir" : "TestEnvExample",
    
    # Devlib module required (or not required)
    'modules' : [ "cpufreq", "cgroups" ],
    #"exclude_modules" : [ "hwmon" ],
    
    # Local installation path used for kernel/dtb installation on target
    # The specified path MUST be accessible from the board, e.g.
    # - JUNO/TC2: it can be the mount path of the VMESD disk image
    # - Other board: it can be a TFTP server path used by the board bootloader
    # "tftp"  : {
    #    "folder"    : "/var/lib/tftpboot",
    #    "kernel"    : "kern.bin",
    #    "dtb"       : "dtb.bin",
    # },

}

In [8]:
from env import TestEnv

# Initialize a test environment using the provided configuration
te = TestEnv(conf)

2018-07-10 16:13:14,207 INFO    : root         : Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
2018-07-10 16:13:14,234 INFO    : root         : Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt
2018-07-10 16:13:14,678 INFO    : TestEnv      : Using base path: /home/wz/dev/linaro-dev/lisa
2018-07-10 16:13:14,680 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-07-10 16:13:14,681 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-07-10 16:13:14,682 WARNING : TestEnv      :    /home/wz/dev/linaro-dev/lisa/results/TestEnvExample
2018-07-10 16:13:14,684 INFO    : TestEnv      : External tools using:
2018-07-10 16:13:14,686 INFO    : TestEnv      :    ANDROID_HOME: /home/wz/dev/program/android_sdk_ndk/android-sdk-linux
2018-07-10 16:13:14,687 INFO    : TestEnv      :    CATAPULT_HOME: /home/wz/dev/linaro-dev/lisa/tools/catapult
2018-07-10 16:13:14,689 INFO    : TestEnv      : Loading board:


## Attributes

The initialization of the test environment pre-initialize some useful<br>
environment variables which are available to write test cases.

These are some of the information available via the TestEnv object.

In [9]:
# The complete configuration of the target we have configured
print json.dumps(te.conf, indent=4)

{
    "ftrace": {
        "buffsize": 10240, 
        "events": [
            "cpu_idle", 
            "sched_switch"
        ]
    }, 
    "modules": [
        "cpufreq", 
        "cgroups"
    ], 
    "results_dir": "TestEnvExample", 
    "platform": "android", 
    "board": "sp9863a", 
    "device": "SC98631A10183230316", 
    "__features__": [], 
    "tools": [
        "cpuidle_sampling.sh"
    ]
}


In [10]:
# Last configured kernel and DTB image
# print te.kernel
# print te.dtb

In [11]:
# The IP and MAC address of the target
# print te.ip
# print te.mac

In [12]:
# A full platform descriptor
print json.dumps(te.platform, indent=4)

{
    "kernel": {
        "version_number": 4, 
        "sha1": null, 
        "major": 4, 
        "parts": [
            4, 
            4, 
            83
        ], 
        "version": "11 SMP PREEMPT Tue Jul 10 16:02:11 CST 2018", 
        "rc": null, 
        "release": "4.4.83+", 
        "minor": 83
    }, 
    "abi": "arm64", 
    "freqs": {
        "big": [
            768000, 
            1050000, 
            1225000, 
            1400000, 
            1500000, 
            1570000
        ], 
        "little": [
            768000, 
            884000, 
            1000000, 
            1100000, 
            1200000
        ]
    }, 
    "nrg_model": {
        "big": {
            "cluster": {
                "nrg_max": 179
            }, 
            "cpu": {
                "cap_max": 1024, 
                "nrg_max": 352
            }
        }, 
        "little": {
            "cluster": {
                "nrg_max": 1
            }, 
            "cpu": {
              

In [13]:
# A pre-created folder to host the tests results generated using this
# test environment, notice that the suite could add additional information
# in this folder, like for example a copy of the target configuration
# and other target specific collected information
te.res_dir

'/home/wz/dev/linaro-dev/lisa/results/TestEnvExample'

In [14]:
# The working directory on the target
te.workdir

'/data/local/tmp/devlib-target'

## Functions

Some methods are also exposed to test developers which could be used to easy
the creation of tests.

These are some of the methods available:

In [15]:
# Calibrate RT-App (if required) and get the most updated calibration value
te.calibration()

In [16]:
# Generate a JSON file with the complete platform description
te.platform_dump(dest_dir='/tmp')

({'abi': 'arm64',
  'clusters': {'big': [4, 5, 6, 7], 'little': [0, 1, 2, 3]},
  'cpus_count': 8,
  'freqs': {'big': [768000, 1050000, 1225000, 1400000, 1500000, 1570000],
   'little': [768000, 884000, 1000000, 1100000, 1200000]},
  'kernel': {'major': 4,
   'minor': 83,
   'parts': (4, 4, 83),
   'rc': None,
   'release': '4.4.83+',
   'sha1': None,
   'version': '11 SMP PREEMPT Tue Jul 10 16:02:11 CST 2018',
   'version_number': 4},
  'nrg_model': {u'big': {u'cluster': {u'nrg_max': 179},
    u'cpu': {u'cap_max': 1024, u'nrg_max': 352}},
   u'little': {u'cluster': {u'nrg_max': 1},
    u'cpu': {u'cap_max': 782, u'nrg_max': 181}}},
  'os': 'android',
  'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]},
 '/tmp/platform.json')

In [17]:
# Force a reboot of the target (and wait specified [s] before reconnect)
# te.reboot(reboot_time=60, ping_time=15)

In [18]:
# Resolve a MAC address into an IP address
# te.resolv_host(host='00:02:F7:00:5A:5B')

In [19]:
# Copy the specified file into the TFTP server folder defined by configuration
#te.tftp_deploy('/etc/group')

In [20]:
#!ls -la /var/lib/tftpboot

# Access to the devlib API

A special TestEnv attribute is <b>target</b>, which represents a <b>devlib instance</b>.
Using the target attribute we can access to the full set of devlib provided
functionalities. Which are summarized in the following sections.

## Remotes commands execution

In [21]:
# Run a command on the target
te.target.execute("echo -n 'Hello Test Environment'", as_root=False)

'Hello Test Environment'

In [22]:
# Spawn a command in background on the target
logging.info("Spawn a task which will run for a while...")
process = te.target.kick_off("sleep 10", as_root=True)

2018-07-10 16:13:25,753 INFO    : root         : Spawn a task which will run for a while...


In [23]:
output = te.target.execute("ps")
print '\n'.join(output.splitlines())

USER           PID  PPID     VSZ    RSS WCHAN            ADDR S NAME                       
root             1     0   12984   2824 0                   0 S init
root             2     0       0      0 0                   0 S [kthreadd]
root             3     2       0      0 0                   0 S [ksoftirqd/0]
root             5     2       0      0 0                   0 S [kworker/0:0H]
root             7     2       0      0 0                   0 S [rcu_preempt]
root             8     2       0      0 0                   0 S [rcu_sched]
root             9     2       0      0 0                   0 S [rcu_bh]
root            10     2       0      0 0                   0 S [migration/0]
root            11     2       0      0 0                   0 S [watchdog/0]
root            12     2       0      0 0                   0 S [watchdog/1]
root            13     2       0      0 0                   0 S [migration/1]
root            14     2       0      0 0                   0 S [ksoft

Notice that _the Shell PID is always the same_ for all commands we execute.<br>
This is due to devlib ensuring to keep a **persistent connection** with the target device.

## Running custom scripts

In [24]:
my_script = te.target.get_installed("cpuidle_sampling.sh")
print my_script

/data/local/tmp/bin/cpuidle_sampling.sh


In [25]:
output = te.target.execute(my_script, as_root=True)
output.splitlines()

['#         Time          idle0          idle1 ',
 '    1325713246            136             47 ',
 '    1325713247             89             33 ',
 '    1325713248            105             30 ']

Notice that the output is returned as a **list of lines**. This provides a useful base for post-processing the output of that command.

In [26]:
# We can also use "notebook embedded" scripts
# my_script = " \
# for I in $(seq 3); do \
#     grep '' /sys/devices/system/cpu/cpu*/cpufreq/stats/time_in_stats | \
#     sed -e 's|/sys/devices/system/cpu/cpu||' -e 's|/cpufreq/scaling_governor:| |' \
#     sleep 1 \
# done \
# "

In [27]:
# print te.target.execute(my_script)

## Access to target specific attributes

In [28]:
# Acces to many target specific information
print "ABI                 : ", te.target.abi
print "big Core Family     : ", te.target.big_core
print "LITTLE Core Family  : ", te.target.little_core
print "CPU's Clusters IDs  : ", te.target.core_clusters
print "CPUs type           : ", te.target.core_names

ABI                 :  arm64
big Core Family     :  A55_1
LITTLE Core Family  :  A55_0
CPU's Clusters IDs  :  [0, 0, 0, 0, 1, 1, 1, 1]
CPUs type           :  [u'A55_0', u'A55_0', u'A55_0', u'A55_0', u'A55_1', u'A55_1', u'A55_1', u'A55_1']


In [29]:
# Access to big.LITTLE specific information
print "big CPUs IDs        : ", te.target.bl.bigs
print "LITTLE CPUs IDs     : ", te.target.bl.littles
print "big CPUs freqs      : {}".format(te.target.bl.get_bigs_frequency())
print "big CPUs governor   : {}".format(te.target.bl.get_bigs_governor())

big CPUs IDs        :  [4, 5, 6, 7]
LITTLE CPUs IDs     :  [0, 1, 2, 3]
big CPUs freqs      : 1570000
big CPUs governor   : schedutil


## Modules usage example: CPUFreq

In [30]:
# You can use autocompletion to have a look at the supported method for a
# specific module
te.target.cpufreq #.get_all_governors()

In [31]:
# Get goverors available for CPU0
te.target.cpufreq.list_governors(0)

['interactive',
 'ondemand',
 'userspace',
 'powersave',
 'performance',
 'schedutil']

In [32]:
# Set the "ondemand" governor
te.target.cpufreq.set_governor(0, 'ondemand')

In [33]:
# Check governor tunables
te.target.cpufreq.get_governor_tunables(0)

{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '2000000',
 'sampling_rate_min': '10000',
 'up_threshold': '95'}

In [34]:
# Update governor tunables
te.target.cpufreq.set_governor_tunables(0, sampling_rate=2000000)
te.target.cpufreq.get_governor_tunables(0)

{'ignore_nice_load': '0',
 'io_is_busy': '0',
 'powersave_bias': '0',
 'sampling_down_factor': '1',
 'sampling_rate': '2000000',
 'sampling_rate_min': '10000',
 'up_threshold': '95'}

## Modules usage example: CGroups

In [35]:
logging.info('%14s - Available controllers:', 'CGroup')
ssys = te.target.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    print '{:10} (hierarchy id: {:d}) has {} cgroups'.format(n, h, g)

2018-07-10 16:13:36,741 INFO    : root         :         CGroup - Available controllers:


cpuset     (hierarchy id: 5) has 7 cgroups
cpu        (hierarchy id: 4) has 1 cgroups
cpuacct    (hierarchy id: 1) has 2 cgroups
schedtune  (hierarchy id: 3) has 5 cgroups
memory     (hierarchy id: 2) has 182 cgroups
freezer    (hierarchy id: 6) has 1 cgroups
hugetlb    (hierarchy id: 6) has 1 cgroups


In [36]:
# Get a reference to the CPUSet controller
cpuset = te.target.cgroups.controller('cpuset')

In [37]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
print 'Existing CGropups:'
for cg in cgroups:
    print "   ", cg

Existing CGropups:
    /
    /LITTLE
    /system-background
    /background
    /foreground
    /foreground/boost
    /top-app


In [38]:
# Create a LITTLE partition and check which tunables we have
cpuset_littles = cpuset.cgroup('/LITTLE')
cpuset_littles.get()

{'cpu_exclusive': '0',
 'cpus': '0-3',
 'effective_cpus': '0-3',
 'effective_mems': '0',
 'ls': ' /data/local/tmp/devlib-target/cgroups/devlib_cgh5/LITTLE/cpuset.*',
 'mem_exclusive': '0',
 'mem_hardwall': '0',
 'memory_migrate': '0',
 'memory_pressure': '0',
 'memory_spread_page': '0',
 'memory_spread_slab': '0',
 'mems': '0',
 'notify_on_release': '0',
 'sched_load_balance': '1',
 'sched_relax_domain_level': '-1'}

In [39]:
# Setup CPUs and MEMORY nodes for the LITTLE partition
cpuset_littles.set(cpus=te.target.bl.littles, mems=0)

In [40]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    cpus = attrs['cpus']
    print '{}:{:<15} cpus: {}'.format(cpuset.kind, cgroup.name, cpus)

cpuset:/               cpus: 0-7
cpuset:/LITTLE         cpus: 0-3
cpuset:/system-background cpus: 0-3
cpuset:/background     cpus: 0-1
cpuset:/foreground     cpus: 0-5
cpuset:/foreground/boost cpus: 0-5
cpuset:/top-app        cpus: 0-7


In [41]:
# Methods exists to move tasks in/out and in between groups
# cpuset_littles.add_task()

# Sample energy from the target

In [42]:
# Reset and sample energy counters
# te.emeter.reset()

# Sleep some time
# time.sleep(2)

# Sample energy consumption since last reset
#nrg = te.emeter.sample()
#nrg = json.dumps(te.emeter.sample(), indent=4)
#print "First read: ", nrg

# Sleep some more time
# time.sleep(2)

# Sample again
#nrg = te.emeter.sample()
#nrg = json.dumps(te.emeter.sample(), indent=4)
#print "Second read: ", nrg

# Configure FTrace for a sepcific experiment

In [43]:
# Configure a specific set of events to trace
te.ftrace_conf(
    {                                                                                                                                             
         "events" : [                                                                                                                                            
             "cpu_idle",                                                                                                                                         
             "cpu_capacity",
             "cpu_frequency",
             "sched_switch",
         ],                                                                                                                                                      
         "buffsize" : 10240                                                                                                                                      
    }
)

2018-07-10 16:13:40,570 INFO    : TestEnv      : Enabled tracepoints:
2018-07-10 16:13:40,572 INFO    : TestEnv      :    cpu_idle
2018-07-10 16:13:40,573 INFO    : TestEnv      :    cpu_capacity
2018-07-10 16:13:40,574 INFO    : TestEnv      :    cpu_frequency
2018-07-10 16:13:40,576 INFO    : TestEnv      :    sched_switch


In [44]:
# Start/Stop a FTrace session
te.ftrace.start()
te.target.execute("uname -a")
te.ftrace.stop()

In [45]:
# Collect and visualize the trace
trace_file = os.path.join(te.res_dir, 'trace.dat')
te.ftrace.get_trace(trace_file)
output = os.popen("DISPLAY=:0.0 kernelshark {}".format(trace_file))